In [15]:
import requests
import json

def generate_query(first, skip):
  query = """
  query {
    spaces(first: %s, skip: %s) {
      id
      name
      website
      twitter
    }
  }
  """ % (first, skip)
  return query


def get_all_organizations():
    URL = "https://hub.snapshot.org/graphql"

    all_organizations = []

    has_next_page = True
    skip = 0
    while has_next_page:
        query = generate_query(1000, skip)
        print('.')
        r = requests.post(URL, json={'query': query})
        data = json.loads(r.text)

        organizations = data['data']['spaces']

        if not organizations:
            print('No organizations found')
            if data.get('errors'):
                print(data['errors'])
                if 'The `skip` argument must not be greater than 15000' in data['errors'][0]['message']:
                    break
        all_organizations += organizations

        if len(organizations) < 1000:
            has_next_page = False
        skip += 1000

    all_organizations_df = pd.DataFrame(all_organizations)

    # add snapshotId to the df
    return all_organizations_df

df = get_all_organizations()

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
No organizations found
[{'message': 'The `skip` argument must not be greater than 15000', 'locations': [{'line': 3, 'column': 5}], 'path': ['spaces']}]


In [16]:
# combine twitter and website into an 'additional' column, ignore NaNs
df['additional'] = df[['twitter', 'website']].apply(lambda x: ', '.join(x.dropna()), axis=1)
# drop twitter and website columns
df.drop(['twitter', 'website'], axis=1, inplace=True)
df

,id,name,additional
0,groomi.eth,Art,seeuart
1,3.onara.eth,Office,
2,lowadka.eth,lowadka,https://t.me/pathdegen
3,geekgeek.eth,BAKC,
4,aetdevinz.eth,devinz,HDevinz
...,...,...,...
15995,maskhuman.eth,HumanDAO,
15996,2021519.eth,DAOFamily,
15997,edc3000.eth,People Power,
15998,binance7.eth,binance7,


In [17]:
# add platform = 'snapshot'
df['platform'] = 'snapshot'
# rename id to platform_id
df.rename(columns={'id': 'platform_id'}, inplace=True)
df

,platform_id,name,additional,platform
0,groomi.eth,Art,seeuart,snapshot
1,3.onara.eth,Office,,snapshot
2,lowadka.eth,lowadka,https://t.me/pathdegen,snapshot
3,geekgeek.eth,BAKC,,snapshot
4,aetdevinz.eth,devinz,HDevinz,snapshot
...,...,...,...,...
15995,maskhuman.eth,HumanDAO,,snapshot
15996,2021519.eth,DAOFamily,,snapshot
15997,edc3000.eth,People Power,,snapshot
15998,binance7.eth,binance7,,snapshot


In [18]:
# add website column https://snapshot.org/#/ + platform_id
df['website'] = 'https://snapshot.org/#/' + df['platform_id']
df

,platform_id,name,additional,platform,website
0,groomi.eth,Art,seeuart,snapshot,https://snapshot.org/#/groomi.eth
1,3.onara.eth,Office,,snapshot,https://snapshot.org/#/3.onara.eth
2,lowadka.eth,lowadka,https://t.me/pathdegen,snapshot,https://snapshot.org/#/lowadka.eth
3,geekgeek.eth,BAKC,,snapshot,https://snapshot.org/#/geekgeek.eth
4,aetdevinz.eth,devinz,HDevinz,snapshot,https://snapshot.org/#/aetdevinz.eth
...,...,...,...,...,...
15995,maskhuman.eth,HumanDAO,,snapshot,https://snapshot.org/#/maskhuman.eth
15996,2021519.eth,DAOFamily,,snapshot,https://snapshot.org/#/2021519.eth
15997,edc3000.eth,People Power,,snapshot,https://snapshot.org/#/edc3000.eth
15998,binance7.eth,binance7,,snapshot,https://snapshot.org/#/binance7.eth


In [19]:
# reorder columns
df = df[['platform', 'platform_id', 'name', 'additional', 'website']]
df

,platform,platform_id,name,additional,website
0,snapshot,groomi.eth,Art,seeuart,https://snapshot.org/#/groomi.eth
1,snapshot,3.onara.eth,Office,,https://snapshot.org/#/3.onara.eth
2,snapshot,lowadka.eth,lowadka,https://t.me/pathdegen,https://snapshot.org/#/lowadka.eth
3,snapshot,geekgeek.eth,BAKC,,https://snapshot.org/#/geekgeek.eth
4,snapshot,aetdevinz.eth,devinz,HDevinz,https://snapshot.org/#/aetdevinz.eth
...,...,...,...,...,...
15995,snapshot,maskhuman.eth,HumanDAO,,https://snapshot.org/#/maskhuman.eth
15996,snapshot,2021519.eth,DAOFamily,,https://snapshot.org/#/2021519.eth
15997,snapshot,edc3000.eth,People Power,,https://snapshot.org/#/edc3000.eth
15998,snapshot,binance7.eth,binance7,,https://snapshot.org/#/binance7.eth


In [20]:
# save to csv
df.to_csv('snapshot_deployments.csv', index=False)